# 01 Data Collection

•	Sử dụng API để thu thập dữ liệu thời tiết theo thành phố và thời gian.

•	Lưu dữ liệu ra CSV.

•	Ví dụ lấy dữ liệu Open-Meteo hoặc các nguồn khác.

In [1]:
# 📦 Cài đặt và import thư viện
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime

# 🌍 Cấu hình thành phố và toạ độ
cities = {
    "HaNoi": {"lat": 21.0285, "lon": 105.8542},
    "DaNang": {"lat": 16.0471, "lon": 108.2068},
    "HoChiMinh": {"lat": 10.7626, "lon": 106.6602}
}

# Hàm thu thập dữ liệu từ API Open-Meteo
def fetch_weather_for_city(city_name, lat, lon):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ",".join([
            "temperature_2m",
            "rain",
            "relative_humidity_2m",
            "wind_speed_10m",
            "cloud_cover",
            "precipitation"
        ]),
        "timezone": "Asia/Bangkok"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Chuyển dữ liệu về DataFrame
    df = pd.DataFrame({
        "time": data["hourly"]["time"],
        "temperature_2m": data["hourly"]["temperature_2m"],
        "rain": data["hourly"]["rain"],
        "relative_humidity_2m": data["hourly"]["relative_humidity_2m"],
        "wind_speed_10m": data["hourly"]["wind_speed_10m"],
        "cloud_cover": data["hourly"]["cloud_cover"],
        "precipitation": data["hourly"]["precipitation"]
    })
    
    df["city"] = city_name
    return df

# Thu thập dữ liệu từ tất cả thành phố
all_data = pd.DataFrame()
for city, coords in cities.items():
    print(f"Thu thập dữ liệu cho {city}...")
    df_city = fetch_weather_for_city(city, coords["lat"], coords["lon"])
    all_data = pd.concat([all_data, df_city], ignore_index=True)

# Chuẩn hóa tên cột cho bước tiền xử lý
all_data["time"] = pd.to_datetime(all_data["time"])
all_data["date"] = all_data["time"].dt.date
all_data["hour"] = all_data["time"].dt.hour

# 🧾 Đổi tên cột
all_data = all_data.rename(columns={
    "temperature_2m": "temp",
    "rain": "rain_amount",
    "relative_humidity_2m": "humidity",
    "wind_speed_10m": "wind",
    "cloud_cover": "cloud",
    "precipitation": "precip"
})

# 🔎 Chọn và sắp xếp lại cột
all_data = all_data[[
    "date", "hour", "temp", "rain_amount", "humidity", 
    "wind", "cloud", "precip", "city"
]]

# Lưu ra CSV
csv_filename = "D:\weather-tourism-project\data\weather_data_raw.csv"
all_data.to_csv(csv_filename, index=False)
print(f" Dữ liệu đã được lưu ra {csv_filename}")

# Lưu vào MongoDB
# Chuyển cột 'date' sang dạng chuỗi để tránh lỗi serialize
all_data["date"] = all_data["date"].astype(str)

client = MongoClient("mongodb://localhost:27017/")
db = client["weather_db"]
collection = db["raw_weather"]
collection.insert_many(all_data.to_dict("records"))
print(" Dữ liệu đã được lưu vào MongoDB")


<>:76: SyntaxWarning: invalid escape sequence '\w'
<>:76: SyntaxWarning: invalid escape sequence '\w'
C:\Windows\Temp\ipykernel_23728\311486649.py:76: SyntaxWarning: invalid escape sequence '\w'
  csv_filename = "D:\weather-tourism-project\data\weather_data_raw.csv"


Thu thập dữ liệu cho HaNoi...
Thu thập dữ liệu cho DaNang...
Thu thập dữ liệu cho HoChiMinh...
 Dữ liệu đã được lưu ra D:\weather-tourism-project\data\weather_data_raw.csv
 Dữ liệu đã được lưu vào MongoDB
